In [6]:
import gymnasium as gym

env = gym.make("Ant-v4", render_mode="human")  # replace with HalfCheetah-v4 or Walker2d-v4
obs = env.reset()

for _ in range(1000):
    env.render()
    action = env.action_space.sample()  # random actions
    obs, reward, terminated, truncated, info = env.step(action)

    done = terminated or truncated
    if done:
        obs = env.reset()

env.close()


/opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(
/opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [16]:
env = gym.make("Ant-v4", render_mode="human")
obs, _ = env.reset()

for _ in range(1000):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

    # Access mujoco data
    sim = env.unwrapped.data  
    model = env.unwrapped.model  

    # Body masses and COM positions
    masses = model.body_mass  # shape (num_bodies,)
    xpos = sim.xipos           # (num_bodies, 3), center of mass of each body in world frame

    # Full system CoM (mass-weighted average)
    com = np.sum(masses[:, None] * xpos, axis=0) / np.sum(masses)
    print("Center of Mass:", com)

    if terminated or truncated:
        obs, _ = env.reset()

env.close()

Center of Mass: [ 0.04056044 -0.06237463  0.79031189]
Center of Mass: [ 0.04331444 -0.05870636  0.75422985]
Center of Mass: [ 0.0471854  -0.05920397  0.69930905]
Center of Mass: [ 0.06365702 -0.1026131   0.68143057]
Center of Mass: [ 0.08089926 -0.14738896  0.64080463]
Center of Mass: [ 0.09811974 -0.19215215  0.57565984]
Center of Mass: [ 0.11536264 -0.23692869  0.48597843]
Center of Mass: [ 0.12758061 -0.28557458  0.44105203]
Center of Mass: [ 0.13781678 -0.32859126  0.52751251]
Center of Mass: [ 0.13947565 -0.37112393  0.66414185]
Center of Mass: [ 0.14171975 -0.41359747  0.77392832]
Center of Mass: [ 0.14412008 -0.45605647  0.85918702]
Center of Mass: [ 0.14650398 -0.49851009  0.9199153 ]
Center of Mass: [ 0.14883792 -0.54095888  0.95612895]
Center of Mass: [ 0.01570446 -0.03039031  0.79812984]
Center of Mass: [ 0.02386544 -0.03369131  0.75784646]
Center of Mass: [ 0.03196295 -0.03709604  0.69301964]
Center of Mass: [ 0.0401273  -0.04046616  0.6036599 ]
Center of Mass: [ 0.04909951

/opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [15]:
# Linear velocity of each body’s CoM
vels = sim.xvelp   # shape (num_bodies, 3)

# Mass-weighted average = total CoM velocity
com_vel = np.sum(masses[:, None] * vels, axis=0) / np.sum(masses)
print("CoM Velocity:", com_vel)


AttributeError: 'mujoco._structs.MjData' object has no attribute 'xvelp'